In [1]:
from fake_useragent import UserAgent
import requests

ua = UserAgent()

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.f

In [2]:
from newspaper import Article
from queue import Queue
from urllib.parse import quote
from unidecode import unidecode

def get_date(load):
    try:
        date = re.findall(
            '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?', load
        )
        return '%s-%s-%s' % (date[2], date[0], date[1])
    except Exce:
        return False


def run_parallel_in_threads(target, args_list):
    globalparas = []
    result = Queue()

    def task_wrapper(*args):
        result.put(target(*args))

    threads = [
        threading.Thread(target = task_wrapper, args = args)
        for args in args_list
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())
    globalparas = list(filter(None, globalparas))
    return globalparas


def get_article(link, news, date):
    article = Article(link)
    article.download()
    article.parse()
    article.nlp()
    lang = 'ENGLISH'
    if len(article.title) < 5 or len(article.text) < 5:
        lang = 'INDONESIA'
        print('found BM/ID article')
        article = Article(link, language = 'id')
        article.download()
        article.parse()
        article.nlp()
    return {
        'title': article.title,
        'url': link,
        'authors': article.authors,
        'top-image': article.top_image,
        'text': article.text,
        'keyword': article.keywords,
        'summary': article.summary,
        'news': news,
        'date': date,
        'language': lang,
    }

In [3]:
NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT = 0

GOOGLE_NEWS_URL = 'https://www.google.com.my/search?q={}&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{}%2Ccd_max%3A{}&tbm=nws&start={}'

def forge_url(q, start, year_start, year_end):
    global NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT
    NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT += 1
    return GOOGLE_NEWS_URL.format(
        q.replace(' ', '+'), str(year_start), str(year_end), start
    )

In [4]:
num_articles_index = 0
url = forge_url('america', num_articles_index, '2005', '2021')
url

'https://www.google.com.my/search?q=america&source=lnt&tbs=cdr%3A1%2Ccd_min%3A2005%2Ccd_max%3A2021&tbm=nws&start=0'

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [42]:
response = requests.get(url, headers = headers, timeout = 60)

In [12]:
soup = BeautifulSoup(response.content, 'html.parser')
style="text-decoration:none;display:block"

In [34]:
soup.find_all('div', {'class': 'XTjFC WF4CUc'})[0]

'America Magazine'

In [27]:
[dateparser.parse(v.text) for v in soup.find_all('span', {'class': 'WG9SHc'})]

[datetime.datetime(2010, 1, 2, 0, 0),
 datetime.datetime(2020, 7, 4, 16, 59, 12, 389201),
 datetime.datetime(2020, 7, 11, 16, 59, 12, 390981),
 datetime.datetime(2020, 6, 27, 16, 59, 12, 392462),
 datetime.datetime(2015, 9, 22, 0, 0),
 datetime.datetime(2020, 7, 4, 16, 59, 12, 397885),
 datetime.datetime(2020, 6, 18, 16, 59, 12, 400063),
 datetime.datetime(2019, 4, 9, 0, 0),
 datetime.datetime(2020, 5, 18, 0, 0),
 datetime.datetime(2020, 6, 18, 16, 59, 12, 409616)]

In [28]:
import dateparser

str(dateparser.parse('2 weeks ago'))

'2020-07-04 16:59:46.017554'

In [49]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
import dateparser

def extract_links(content):
    soup = BeautifulSoup(content, 'html.parser')
    # return soup
    today = datetime.now().strftime('%m/%d/%Y')
    links_list = [v.attrs['href'] for v in soup.find_all('a', {'style': 'text-decoration:none;display:block'})]
    dates_list = [v.text for v in soup.find_all('span', {'class': 'WG9SHc'})]
    sources_list = [v.text for v in soup.find_all('div', {'class': 'XTjFC WF4CUc'})]
    output = []
    for (link, date, source) in zip(links_list, dates_list, sources_list):
        try:
            date = str(dateparser.parse(date))
        except:
            pass
        output.append((link, date, source))
    return output

In [50]:
r = extract_links(response.content)
r

[('https://www.americamagazine.org/',
  '2010-01-02 00:00:00',
  'America Magazine'),
 ('https://www.theatlantic.com/ideas/archive/2020/07/america-land-pathetic/613747/',
  '2020-07-04 17:04:57.632971',
  'The Atlantic'),
 ('https://www.nytimes.com/2020/07/07/world/europe/american-passport-privilege-coronavirus.html',
  '2020-07-11 17:04:57.634208',
  'The New York Times'),
 ('https://www.theatlantic.com/international/archive/2020/06/america-image-power-trump/613228/',
  '2020-06-27 17:04:57.635410',
  'The Atlantic'),
 ('https://www.bbc.com/news/world-us-canada-34331761',
  '2015-09-22 00:00:00',
  'BBC News'),
 ('https://www.theatlantic.com/ideas/archive/2020/07/when-trumps-understanding-of-the-world-is-flipped-on-its-head/613840/',
  '2020-07-04 17:04:57.639417',
  'The Atlantic'),
 ('https://www.theatlantic.com/ideas/archive/2020/06/why-americas-institutions-are-failing/613078/',
  '2020-06-18 17:04:57.640774',
  'The Atlantic'),
 ('https://www.pewsocialtrends.org/2019/04/09/race-i